# Imports

In [5]:
import pandas as pd
import numpy as np
np.random.seed(0)
from JobsDb import JobsDb
db = JobsDb()
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence
from nltk import word_tokenize
from gensim.models import word2vec

# Load the Data

In [6]:
df = db.load_table_as_df('jobs')
db.close()
df.head()

,id,title,url,description
0,1,Psychiatrist (Per Diem) - #201222-4667HD,https://www.careerjet.com/jobad/us61b2e1c18a4d...,\n \n shall strive to be a global leader i...
1,2,Professor of Osteopathic Manipulative Medicine,https://www.careerjet.com/jobad/us0f769e5a939c...,\nPosition Details Position Title Professor o...
2,3,Amazon Warehouse Assistant - Immediately Hiring,https://www.careerjet.com/jobad/usba95a3670331...,\n \n Hourly pay rate: $15.00 *The base pa...
3,4,Non-QM Underwriter,https://www.careerjet.com/jobad/usb59eda9438ed...,\nJob Description We looking to add an experi...
4,5,"project manager, accounting systems",https://www.careerjet.com/jobad/us71cfd31d23d6...,\njob description job summary: Project Manag...


# Tag the data

In [7]:
df['target'] = df['title'].apply(lambda x: 'data' in x.lower())

# Extract feature and target

In [8]:
y = pd.get_dummies(df['target']).values

In [9]:
tokenizer = text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(df['description']))
list_tokenized_descriptions = tokenizer.texts_to_sequences(df['description'])
X_t = sequence.pad_sequences(list_tokenized_descriptions, maxlen=500)

# Build baseline model

In [6]:
model = Sequential()

In [7]:
embedding_size = 128
model.add(Embedding(20000, embedding_size))
model.add(LSTM(25, return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [8]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         2560000   
_________________________________________________________________
lstm (LSTM)                  (None, None, 25)          15400     
_________________________________________________________________
global_max_pooling1d (Global (None, 25)                0         
_________________________________________________________________
dropout (Dropout)            (None, 25)                0         
_________________________________________________________________
dense (Dense)                (None, 50)                1300      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1

In [10]:
model.fit(X_t, y, epochs=3, batch_size=32, validation_split=0.1)

Epoch 1/3
539/539 [==============================] - 329s 610ms/step - loss: 0.1329 - accuracy: 0.9488 - val_loss: 0.0085 - val_accuracy: 0.9979
Epoch 2/3
539/539 [==============================] - 354s 657ms/step - loss: 0.0291 - accuracy: 0.9937 - val_loss: 0.0057 - val_accuracy: 0.9990
Epoch 3/3
539/539 [==============================] - 314s 582ms/step - loss: 0.0181 - accuracy: 0.9954 - val_loss: 0.0202 - val_accuracy: 0.9937


In [13]:
model.save('../model/baseline')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ../model/baseline/assets


# Load model

In [3]:
from tensorflow import keras
model = keras.models.load_model('../model/baseline')
model.

# Predict classification on an example

In [27]:
doc = df['description'][2000]
embedded_doc = X_t[2000]
embedded_doc

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    6,    6,
         44,   57,  160,   42,   54,   42,  661, 1051,  113,   57,  182,
        636,  428, 1074, 1070, 1071,  198,  696, 10

In [26]:
model.predict_proba(embedded_doc)

array([[0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.5883486 , 0.41165137],
       [0.

In [28]:
import eli5
from eli5.lime import TextExplainer

te = TextExplainer(random_state=42)
te.fit(embedded_doc, model.predict_proba)
te.show_prediction(target_names=model.target_names)

AttributeError: 'numpy.ndarray' object has no attribute 'lower'